In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets


In [2]:
class VGG_net(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(VGG_net, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers()

        self.fcs = nn.Sequential(
            nn.Linear(512, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fcs(x)
        return x

    def create_conv_layers(self):
        layers = []
        in_channels = self.in_channels

        # Simplified VGG configuration for MNIST
        vgg_config = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M']

        for x in vgg_config:
            if isinstance(x, int):
                out_channels = x
                layers += [
                    nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.BatchNorm2d(x),
                    nn.ReLU()
                ]
                in_channels = x
            elif x == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]
        
        return nn.Sequential(*layers)

def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n")


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
BATCH_SIZE = 64
EPOCHS = 3
LEARNING_RATE = 0.01

In [ ]:
# Data Preprocessing
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [14]:
# Model
model = VGG_net().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training and Testing
for epoch in range(1, EPOCHS + 1):
     train(model, device, train_loader, optimizer, criterion, epoch)
     test(model, device, test_loader, criterion)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.271098
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.678319
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.888140
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.936469
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.808881
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.171300
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.423635
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.431500
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.298928
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.379379

Test set: Average loss: 0.0037, Accuracy: 9434/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.433563
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.203395
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.320508
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.297648
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.061809
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.112095
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.254387
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.050161
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.323499
T